<a href="https://colab.research.google.com/github/TheCaveOfAdullam/study3/blob/main/1D_CNN_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import LabelEncoder

In [3]:
# 기본 경로 설정
base_dir = '/content/drive/MyDrive/ship_data'
categories = ['normal', 'fault_BB', 'fault_RI', 'fault_SM']

# 데이터 로드 및 전처리 함수 정의
def load_data(base_dir, split):
    X = []
    y = []
    split_dir = os.path.join(base_dir, split)
    for category in categories:
        category_dir = os.path.join(split_dir, category)
        for file in os.listdir(category_dir):
            file_path = os.path.join(category_dir, file)
            data = pd.read_csv(file_path, header=None).values
            data = pd.to_numeric(data.flatten(), errors='coerce').reshape(-1, data.shape[1])
            data = np.nan_to_num(data).astype('float32')  # NaN 값을 0으로 대체하고, float32로 변환
            X.append(data)
            y.append(category)
    return np.array(X), np.array(y)

# 데이터 로드
X_train, y_train = load_data(base_dir, 'train')
X_val, y_val = load_data(base_dir, 'validation')
X_test, y_test = load_data(base_dir, 'test')

# 데이터 차원 변경 (CNN 입력 형식에 맞게)
X_train = np.expand_dims(X_train, axis=-1)
X_val = np.expand_dims(X_val, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

# 레이블 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)

# 원-핫 인코딩
y_train_categorical = tf.keras.utils.to_categorical(y_train_encoded)
y_val_categorical = tf.keras.utils.to_categorical(y_val_encoded)
y_test_categorical = tf.keras.utils.to_categorical(y_test_encoded)

In [4]:
# 모델 정의
model = models.Sequential()

# Conv1 레이어
model.add(layers.Conv1D(filters=64, kernel_size=16, strides=16, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
# Pool1 레이어
model.add(layers.MaxPooling1D(pool_size=2, strides=2))

# Conv2 레이어
model.add(layers.Conv1D(filters=32, kernel_size=3, strides=1, activation='relu'))

# Conv3 레이어
model.add(layers.Conv1D(filters=64, kernel_size=5, strides=1, activation='relu'))

# Conv4 레이어
model.add(layers.Conv1D(filters=128, kernel_size=5, strides=1, activation='relu'))

# Pool2 레이어
model.add(layers.MaxPooling1D(pool_size=2, strides=2))

# Flatten 레이어
model.add(layers.Flatten())

# FC1 레이어
model.add(layers.Dense(units=5000, activation='relu'))

# FC2 레이어
model.add(layers.Dense(units=1000, activation='relu'))

# Output 레이어
model.add(layers.Dense(len(categories), activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 요약 출력
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 750, 64)           2112      
                                                                 
 max_pooling1d (MaxPooling1  (None, 375, 64)           0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 373, 32)           6176      
                                                                 
 conv1d_2 (Conv1D)           (None, 369, 64)           10304     
                                                                 
 conv1d_3 (Conv1D)           (None, 365, 128)          41088     
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 182, 128)          0         
 g1D)                                                   

In [5]:
# 모델 훈련
history = model.fit(X_train, y_train_categorical, epochs=10, validation_data=(X_val, y_val_categorical))

Epoch 1/10
219/219 [==============================] - 109s 493ms/step - loss: 1.3030 - accuracy: 0.5113 - val_loss: 0.8733 - val_accuracy: 0.8427
Epoch 2/10
219/219 [==============================] - 109s 496ms/step - loss: 0.7261 - accuracy: 0.7471 - val_loss: 0.3112 - val_accuracy: 0.8717
Epoch 3/10
219/219 [==============================] - 109s 498ms/step - loss: 0.3685 - accuracy: 0.8649 - val_loss: 0.2567 - val_accuracy: 0.8772
Epoch 4/10
219/219 [==============================] - 109s 500ms/step - loss: 0.2097 - accuracy: 0.9166 - val_loss: 0.0681 - val_accuracy: 0.9765
Epoch 5/10
219/219 [==============================] - 110s 503ms/step - loss: 0.0883 - accuracy: 0.9667 - val_loss: 0.0331 - val_accuracy: 0.9930
Epoch 6/10
219/219 [==============================] - 110s 502ms/step - loss: 0.1756 - accuracy: 0.9545 - val_loss: 0.0858 - val_accuracy: 0.9664
Epoch 7/10
219/219 [==============================] - 109s 499ms/step - loss: 0.0979 - accuracy: 0.9672 - val_loss: 0.0295 -

In [6]:
# 모델 평가
test_loss, test_accuracy = model.evaluate(X_test, y_test_categorical)
print(f'Test accuracy: {test_accuracy}')

40/40 [==============================] - 2s 39ms/step - loss: 0.0299 - accuracy: 0.9851
Test accuracy: 0.9851329922676086


In [7]:
# 원본 모델 저장
model.save('model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
# 모델 사이즈 측정
model_size = os.path.getsize('model.h5') / (1024 * 1024)
print(f"Model Size: {model_size:.2f} MB")

Model Size: 1391.09 MB


Deep Transfer Learning Method Based on 1D-CNN for Bearing Fault Diagnosis